# An advanced look at LLM Agents
## Notebook 3: Agents behind the hood
---

Previously, we relied on LangChain to to build the agent. However, this is not necessary, since an agent is nothing more than a fancy while loop. 

**The goal** 

With this notebook you will see what an agent is under the hood, and you can build it based on the LLM output

🌟 So ... let us begin!  

**Contents:**

1. [Exercise 1: New tool format](#1)
2. [Exercise 2: String vs function call response](#2)
3. [Exercise 3: Understanding the agent while-loop](#3)

In [1]:
# Run this cell once, if you have not run the following command previously in the terminal. 
# Afterwards commenting it out  so it does not clutter your notebook.
!pip3 install -r requirements.txt

You should consider upgrading via the '/Users/mariabader/projects/women_in_ai/intro-agents-june2024/venv/bin/python3 -m pip install --upgrade pip' command.


In [2]:
from helper_functions.helper_functions import simple_description_formatter
from helper_functions.tools import my_own_wiki_tool, weather_tool
import pprint
from helper_functions.keys import client
import json


/Users/mariabader/projects/women_in_ai/intro-agents-june2024/venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


---

## The tools

When querying OpneAI's API directly tools are now called functions (see the API documentation on function calling [here](https://platform.openai.com/docs/assistants/tools/function-calling/quickstart)). Functions have to be passed in a specific JSON format, which we will explore below.


#### Exercise 1: New tool format <a id='1'></a>

**TASK:**  
Compile the cell below and then compare the description of the tool `my_own_wiki_tool` to the formatted function description.

In [3]:
# load the tools and format. In plain OpenAI jargon they are called functions.
tools = [my_own_wiki_tool, weather_tool]
function_description = [simple_description_formatter(tool) for tool in tools]

# Store executable functions with their name in dictionary
available_functions = {tool.name: tool for tool in tools}


In [5]:
# Formatted function description of the my_own_wiki_tool tool
pprint.pprint(function_description[0])

# Description baked into my_own_wiki_tool
print(my_own_wiki_tool.description
    )

{'function': {'description': 'A wrapper around Wikipedia. Useful for when you '
                             'need to answer general questions about people, '
                             'places, companies, facts, historical events, or '
                             'other subjects. Input should be a search query.',
              'name': 'wikipedia',
              'parameters': {'properties': {'query': {'description': 'Input '
                                                                     'search '
                                                                     'query',
                                                      'type': 'string'}},
                             'required': ['query'],
                             'type': 'object'}},
 'type': 'function'}
A wrapper around Wikipedia. Useful for when you need to answer general questions about people, places, companies, facts, historical events, or other subjects. Input should be a search query.


---

## The LLM response

The LLM can respond with two types of answers:
* a **string** that answers the question, 
* a **function-call** object, which contains information on which function to call with which arguments.

The second one can be used to execute function calls.

#### Exercise 2: String vs function call response <a id='2'></a>

**TASK:**  
Compile both questions, and compare the answers. Do you understand the difference?


In [6]:
system_prompt = "You are a friendly, helpful assistant. Your goal is to answer the questions in a concise, but conversational manner."

questions = ["what is the meaning of life?","How many people live in Paris?"]

for question in questions:
  messages = [
      {"role": "system", "content": system_prompt},
      {"role": "user", "content": question}
    ]

  
  response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    tools = function_description,
    messages=messages, 
    )

  print(f"Question: {question}")
  print(f"Answer: {response.choices[0].message.content}")
  print(f"Function call: {response.choices[0].message.tool_calls}\n")



Question: what is the meaning of life?
Answer: The meaning of life is a philosophical question that varies depending on individual beliefs and perspectives. It is often seen as a profound and subjective concept related to the purpose, significance, and value of human existence. Different philosophies, religions, and cultures offer various interpretations of the meaning of life.
Function call: None

Question: How many people live in Paris?
Answer: None
Function call: [ChatCompletionMessageToolCall(id='call_RyXZGvFS0VcKROJJNq3x3n3n', function=Function(arguments='{"query":"Population of Paris"}', name='wikipedia'), type='function')]



---

## The Agent - a fancy while loop

While the LLM requests function calls we 
* **extract** the **name and arguments** to be called from the initial LLM response,
* **execute** the **function calls**,
* **store** the **output of the function** in the messages object,
* invoke the LLM again, until no function call are requested.

For more details, you can also check out this [OpenAI function calling guide](https://platform.openai.com/docs/guides/function-calling).

#### Exercise 3: Understanding the agent while-loop <a id='3'></a>

**TASK:**  
Complete the code below, then compile the question and investigate the output. Do you understand what you see?

In [7]:
question = "which city is bigger: Paris or Munich?"

messages = [
      {"role": "system", "content": system_prompt},
      {"role": "user", "content": question}
    ]

response = client.chat.completions.create(
  model="gpt-3.5-turbo",
  tools = function_description,
  messages=messages, 
  )

print('==== Initial LLM response ====')
print(f"Answer: {response.choices[0].message.content}")
print(f"Function call: {response.choices[0].message.tool_calls}\n")

# while the response requests function calls
while response.choices[0].message.tool_calls:
    
  # store response message with all function calls
  response_message = response.choices[0].message
  messages.append(response_message)

  # execute each tool individually
  for tool_call in response.choices[0].message.tool_calls:
    print('==== Function call ====')

    # function name and arguments
    function_name = tool_call.function.name
    function_args = json.loads(tool_call.function.arguments)
    print(f'Calling function "{function_name}" with arguments {function_args}.')

    # execute function call 
    function_response = available_functions[function_name].invoke(function_args)
    print(f'Function call response:\n{function_response}\n')

    # append function response to messages
    messages.append({
        "tool_call_id":tool_call.id, 
        "role": "tool", 
        "name": function_name, 
        "content": function_response
    })
    
  # get a new response from LLM
  response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    tools = function_description,
    messages=messages, 
  )

  print('== Intermediate LLM response ==')
  print(f"Answer: {response.choices[0].message.content}")
  print(f"Function call: {response.choices[0].message.tool_calls}\n")

print('==== Final LLM response ====')
print("Question: ", question)
print(f"Answer: {response.choices[0].message.content}")
print(f"Function call: {response.choices[0].message.tool_calls}\n")

==== Initial LLM response ====
Answer: None
Function call: [ChatCompletionMessageToolCall(id='call_lGiJcKMlKcA4R9cUWCflL35N', function=Function(arguments='{"query": "Paris"}', name='wikipedia'), type='function'), ChatCompletionMessageToolCall(id='call_m36ccyXwfjmd7AQHxskLsnhh', function=Function(arguments='{"query": "Munich"}', name='wikipedia'), type='function')]

==== Function call ====
Calling function "wikipedia" with arguments {'query': 'Paris'}.
Function call response:
Page: Paris
Summary: Paris is the capital and largest city of France. With an official estimated population of 2,102,650 residents as of 1 January 2023 in an area of more than 105 km2 (41 sq mi), Paris is the fourth-largest city in the European Union and the 30th most densely populated city in the world in 2022. Since the 17th century, Paris has been one of the world's major centres of finance, diplomacy, commerce, culture, fashion, and gastronomy. For its leading role in the arts and sciences, as well as its early

🌟 Congratulations - you've finished the workshop